In [55]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [56]:
import re
import numpy as np

In [57]:
import pandas as pd
data = pd.read_csv('tweet_emotions1.csv')
data.head(10)

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,@charviray Charlene my love. I miss you
9,1956969172,sadness,@kelcouch I'm sorry at least it's Friday?


In [58]:
data['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [59]:
data.isnull().sum()

tweet_id     0
sentiment    0
content      0
dtype: int64

In [60]:
data['sentiment'].nunique()

13

***

**Pre-Processing**

In [61]:
def remove_id(row):
    content =' '.join(word for word in row.split() if not word.startswith('@'))
    content = content.lower()
    return content
# Apply the function to the 'context' column
data['content'] = data['content'].apply(remove_id)

In [62]:
def remove_urls(text):
    word = re.compile(r"http\S+|www\S+")
    word = re.compile(r"[.,!?#&;/]")
    return word.sub("", text)

# Apply the remove_urls function to the 'content' column
data['content'] = data['content'].apply(remove_urls)

In [ ]:
def remove_spaces(text):
    space_pattern = re.compile(r"\s+")
    return space_pattern.sub(" ", text).strip()

data['content'] = data['content'].apply(remove_spaces)

***

**Tokenization , Lemmatization**

In [64]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [65]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adith\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adith\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adith\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [66]:
def fun1(text):
    tokens = word_tokenize(text)
    lm = WordNetLemmatizer()
    lemmatized_tokens = [lm.lemmatize(token) for token in tokens]
    filtered_tokens = [re.sub(r'[^\w\s]', '', token) for token in lemmatized_tokens if token]
    
    return filtered_tokens

data['content'] = data['content'].apply(fun1)


In [67]:
data = data[data['content'].map(len) > 0]
data = data.reset_index(drop=True)

In [68]:
data

,tweet_id,sentiment,content
0,1956967341,empty,"[i, know, i, wa, listenin, to, bad, habit, ear..."
1,1956967666,sadness,"[layin, n, bed, with, a, headache, ughhhhwaiti..."
2,1956967696,sadness,"[funeral, ceremonygloomy, friday]"
3,1956967789,enthusiasm,"[want, to, hang, out, with, friend, soon]"
4,1956968416,neutral,"[we, want, to, trade, with, someone, who, ha, ..."
...,...,...,...
39909,1753918900,happiness,"[succesfully, following, tayla]"
39910,1753919001,love,"[happy, mother, day, all, my, love]"
39911,1753919005,love,"[happy, mother, s, day, to, all, the, mommy, o..."
39912,1753919043,happiness,"[wassup, beautiful, follow, me, peep, out, my,..."


***

**Finding out the Max Length Value**

In [69]:
token_lengths = []

# Iterate over each row in the DataFrame
for index, row in data.iterrows():
    tokens = row['content']
    token_lengths.append(len(tokens))

# Convert the list of token lengths to a DataFrame column
data['token_length'] = token_lengths


In [70]:
list1 = []
for x in data['token_length']:
    list1.append(x)
percentile_95 = np.percentile(token_lengths, 95)
print(percentile_95)

26.0


In [73]:
import statistics as st
print(st.mean(list1))

13.32735381069299


In [72]:
print(max(list1))

44


***

**Feature Extraction**

In [74]:
data1 = data.sample(n = 15000)

In [75]:
data1

,tweet_id,sentiment,content,token_length
6829,1961463950,worry,"[, o, awww, leave, off, shes, ace, shes, one, ...",13
39703,1753871880,neutral,"[lmao, , that, would, have, taken, me, at, lea...",11
30834,1751751523,neutral,"[it, 1111make, a, wish]",4
27281,1695731147,happiness,"[happy, star, war, day, may, the, 4th, be, wit...",10
12872,1963781502,worry,"[aww, thats, too, bad, you, lost, it, though]",8
...,...,...,...,...
3725,1958113021,worry,"[bahhhh, cold, weather, is, making, my, teeth,...",8
32262,1752257523,sadness,"[ducked, off, i, ll, be, back, in, ga, tomorrow]",9
26325,1695376673,neutral,"[currently, uploading, wordpress, this, ll, gi...",16
39324,1753819653,love,"[happy, mother, s, day, mom, you, are, wonderf...",12


In [76]:
from transformers import GPT2Model,GPT2Tokenizer
import torch
model = GPT2Model.from_pretrained('gpt2')

In [78]:
import numpy as np
from transformers import GPT2Tokenizer

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the maximum sequence length for padding
max_length = 30

# Create a new column 'features' in the DataFrame
data1['features'] = None

# Iterate over each row in the DataFrame
for index, row in data1.iterrows():
    tokens = row['content']

    # Truncate or pad the tokens to the maximum length
    tokens = tokens[:max_length] if len(tokens) > max_length else tokens + ['[PAD]'] * (max_length - len(tokens))

    # Convert tokens into token IDs
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Skip empty tokens
    if len(input_ids) == 0:
        continue

    # Convert token IDs into input tensors
    input_ids = torch.tensor(input_ids).unsqueeze(0)

    # Pass the non-empty input tensors through the pre-trained GPT-2 model to extract the features
    with torch.no_grad():
        outputs = model(input_ids)

    features = outputs.last_hidden_state

    # Store the features in the 'features' column for the current row
    data1.at[index, 'features'] = features

In [79]:
data1 = data1.dropna(subset=['features'])

# Convert the 'features' column to a NumPy array
x = np.stack([tensor.numpy() for tensor in data1['features'].values])

***

**Target Encoding**

In [85]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(data1['sentiment'])
label_mapping = {index: label for index, label in enumerate(encoder.classes_)}

In [86]:
label_mapping

{0: 'anger',
 1: 'boredom',
 2: 'empty',
 3: 'enthusiasm',
 4: 'fun',
 5: 'happiness',
 6: 'hate',
 7: 'love',
 8: 'neutral',
 9: 'relief',
 10: 'sadness',
 11: 'surprise',
 12: 'worry'}

In [87]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=13)
y

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

***

**Model**

In [88]:
print(x.shape)
y.shape

(15000, 1, 30, 768)


(15000, 13)

In [89]:
x = np.reshape(x,(15000,30,768))

In [90]:
print(x.shape)
y.shape

(15000, 30, 768)


(15000, 13)

In [91]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.2)

In [93]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(128, input_shape=(30, 768), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(13, activation='softmax'))


In [94]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [95]:
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
375/375 [==============================] - 71s 181ms/step - loss: 2.2167 - accuracy: 0.2065 - val_loss: 2.1522 - val_accuracy: 0.2067
Epoch 2/100
375/375 [==============================] - 70s 187ms/step - loss: 2.1648 - accuracy: 0.2218 - val_loss: 2.1406 - val_accuracy: 0.2520
Epoch 3/100
375/375 [==============================] - 71s 189ms/step - loss: 2.1519 - accuracy: 0.2470 - val_loss: 2.1324 - val_accuracy: 0.2487
Epoch 4/100
375/375 [==============================] - 71s 191ms/step - loss: 2.1299 - accuracy: 0.2520 - val_loss: 2.0921 - val_accuracy: 0.2697
Epoch 5/100
375/375 [==============================] - 71s 189ms/step - loss: 2.0971 - accuracy: 0.2673 - val_loss: 2.0799 - val_accuracy: 0.2727
Epoch 6/100
375/375 [==============================] - 71s 188ms/step - loss: 2.0831 - accuracy: 0.2757 - val_loss: 2.0746 - val_accuracy: 0.2793
Epoch 7/100
375/375 [==============================] - 70s 187ms/step - loss: 2.0697 - accuracy: 0.2793 - val_loss: 2.0691 -

In [96]:
loss, accuracy = model.evaluate(x_test, y_test)
accuracy

94/94 [==============================] - 8s 87ms/step - loss: 3.1143 - accuracy: 0.2693


0.2693333327770233

***

**PreTrained** 

In [107]:
data = data1.sample(n = 50)

In [113]:
x = data['features']
x = np.stack([tensor.numpy() for tensor in data1['features'].values])

In [111]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(data['sentiment'])
label_mapping = {index: label for index, label in enumerate(encoder.classes_)}
from keras.utils import to_categorical
y = to_categorical(y, num_classes=13)
y

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 

In [114]:
import torch
import torch.nn as nn
from transformers import GPT2Model

class GPT2ClassificationModel(nn.Module):
    def __init__(self, num_classes):
        super(GPT2ClassificationModel, self).__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(768, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids):
        outputs = self.gpt2(input_ids)[0]
        pooled_output = torch.mean(outputs, dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

classification_model = GPT2ClassificationModel(13)
# Convert the model to evaluation mode
classification_model.eval()
X_tensor_test = torch.tensor(x_test, dtype=torch.long)

with torch.no_grad():
    logits_test = classification_model(X_tensor_test)

probs_test = nn.functional.softmax(logits_test, dim=1)
predicted_classes_test = torch.argmax(probs_test, dim=1)

# Convert predicted classes to numpy array
y_test_pred = predicted_classes_test.numpy()

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 212336640000 bytes.

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_test_pred))